In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nikhi\\End-To-End-ML-Project\\research'

In [3]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            target_column=schema.name
        )

        return model_evaluation_config

In [ ]:
import pandas as pd
import numpy as np
import joblib

from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from mlProject.utils.common import save_json
from mlProject.entity.config_entity import ModelEvaluationConfig
from mlProject import logger

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        logger.info("Starting model evaluation")

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column] 

        predicted = model.predict(test_x)

        rmse, mae, r2 = self.eval_metrics(test_y, predicted)

        scores = {
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        }

        save_json(
            path=Path(self.config.metric_file_name),
            data=scores
        )

        logger.info(f"Model evaluation completed. RMSE: {rmse}, MAE: {mae}, R2: {r2}")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2026-01-23 11:51:44,694: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-23 11:51:44,699: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-23 11:51:44,709: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-23 11:51:44,712: INFO: common: created directory at: artifacts]
[2026-01-23 11:51:44,716: INFO: common: created directory at: artifacts/model_evaluation]
[2026-01-23 11:51:46,153: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
